In [10]:
import random
import numpy as np

In [2]:
E = EllipticCurve(GF(144169), [1,1])
listepoints = E.points()

In [11]:
def decomp_base2(x):
    y=bin(x)
    return y[2:]

def harmonisation(a, w):
    strNum = str(a)
    a = strNum.zfill(len(strNum)+w-(len(strNum)%w))
    return a

def extract(i,a,w):
    return a[w*i:w*(i+1)]

In [12]:
def algorithme_naif(P,k):
    if k == 0:
        return E(0)
    elif k == 1:
        return P
    else :
        return P + P + algorithme_naif(P,k-2)

In [13]:
def double_aj(P,k):
    if k == 0:
        return E(0)
    elif k == 1:
        return P
    elif k%2 == 0:
        j = k//2
        return double_aj(P+P, j)
    else:
        j = (k-1)//2
        return P + double_aj(P+P, j)

In [14]:
def WNAF_avec_w(P,k,w):
    Q = P
    L = [E(0)]
    a = decomp_base2(k)
    m = L[0]

    #On a deux cas "triviaux" où il est plus simple 
    #de renvoyer à notre deuxième algorithme :
    if len(a)==2 or len(a)==1:
        return double_aj(P,k)

    #Sinon, on le lance :
    #La longueur de a doit etre divisible par w.
    else :
        b = len(a)%w                #Donc si besoin, on ajoute des 0 1/3
        if b!=0:                    # a gauche,ca permet de diviser  2/3
            c = 1 + (len(a)-b)/w    # a en fenetre de tailles w.     3/3
            a = harmonisation(a, c*w)

        #La phase de preparation est terminee. On peut commencer WNAF :
        for i in range(2**w):      #On prepare la liste :
            L.append(Q)            #L = [E(0), P, 2*P, ..., (-1 + 2**w)*P]
            Q = Q + P
        for i in range(len(a)//w): #On divise a en sous-parties de taille w
            n = m
            for j in range(2**w - 1):
                m += n       #Le m est inchange si c'est E(0)       1/5
                             #(a l'etape 1 de l'algorithme, i=0),   2/5
                             #mais sinon, il ne faut pas oublier de 3/5
                             #multiplier par 2**w,                  4/5
                             #pour bien respecter l'echelle !       5/5

            k = extract(i,a,w) #On extrait la 1ère fenetre de taille 1/3
            k = int(k,2)       # w du coefficient en base 2, et on   2/3
                               # convertit cette fenêtre en base 10. 3/3

            k = L[k]     #On prend le point correspondant dans L,   1/5
            m += k       # on l'ajoute à notre calcul. Et si ce     2/5
                         # n'est pas la derniere étape de la boucle 3/5
                         # for, on multipliera par 2^w pour pouvoir 4/5
                         # passer à la sous-partie suivante.        5/5
        return m

In [15]:
def WNAF_sans_w(P,k):
    Q = P
    L = [E(0)]
    m = L[0]
    a = decomp_base2(k)

    #On cherche à diviser a en sous-parties de taille égale.
    #Ce qui sera un problème si a est de longueur indivisible.
    if is_prime(len(a)) is True :
        if len(a)==2 or len(a)==1:
            #Ici, autant retourner à un algorithme plus simple, 1/2
            #On ne perdra pas en efficacité : 2/2
            return double_aj(P,k)

        else :
            #Mais sinon, on utilise une astuce inspirée de doubler-  1/5
            # ajouter. La représentation en base 2 de (k-h)//2       2/5
            # sera de longueur non première, donc on pourra lancer   3/5
            # l'algorithme sans retomber sur le meme probleme        4/5
            # et cela nous ajoute seulement 2 additions.             5/5

            h = int(a[-1],2) #h est egal à 0 ou 1. Il est preferable  1/3
                             # de le definir ainsi, car cela evite    2/3
                             # un if... else qui serait lourd.        3/3
            return h*P + WNAF_sans_w(P + P,(k-h)//2)

    else :
        #On ouvre une fenêtre de taille w. 1/2
        #Il faut seulement trouver un w adéquat : 2/2
        p1 = factor(len(a))[-1][0]
        r1 = factor(len(a))[-1][1]
        w = p1^(r1-1)
        #Ici, on utilise la factorisation de k en nombres premiers!     1/3
        #Notons qu'on aurait pu choisir autre chose pour w, c'est un    2/3
        # parti pris dans l'algorithme qui peut tout à fait être changé 3/3

        #Et maintenant qu'on a un bon w, il nous reste la derniere 1/3
        # partie de l'algorithme, qui est exactement la meme 2/3
        # que celle de l'autre algorithme WNAF ! 3/3
        for i in range(2**w):
            L.append(Q)
            Q = Q + P
        for i in range(len(a)//w):
            n = m
            for j in range(2**w - 1):
                m += n
            k = extract(i,a,w)
            k = int(k,2)
            k = L[k]
            m += k
        return m